In [ ]:
%load_ext autoreload
%autoreload 2
#%pylab inline

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [ ]:
import tensorflow as tf
import numpy as np
import itertools
import keras
import gym

from tqdm import tqdm
from wrappers.envs import AtariStackFrames
from utils.ae import AEConvKeras

In [ ]:
model = keras.models.load_model('models/model_20170209-171440/model_epoch99.h5')

In [ ]:
env = AtariStackFrames(gym.make('MsPacman-v0'), rgb=True)

In [ ]:
def collect_data(env=None, n_episodes=10, max_steps=5000):
    states = []
    for i in tqdm(range(n_episodes)):
        state = env.reset()
        states.append(state)
        terminal = False
        step = 0
        while not terminal and step < max_steps:
            action = env.action_space.sample()
            state, reward, terminal, _ = env.step(action)
            states.append(state)
            step += 1
    return np.array(states)

def run_episode(env=None, agent=None, max_steps=5000):
    state = env.reset()
    total_reward = 0.
    terminal = False
    step = 0
    while not terminal and step < max_steps:
        action = agent.predict(np.expand_dims(state, 0)).argmax(axis=1)
        #action = env.action_space.sample()
        state, reward, terminal, _ = env.step(action)
        total_reward += reward
        step += 1
    return total_reward

In [ ]:
res = []
for i in xrange(20):
    r = run_episode(env, model)
    print r
    res.append(r)
print 'mean: {:.3f}'.format(np.array(res).mean())

## temp

In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [2]:
import keras
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model

Using TensorFlow backend.


In [ ]:
inp = Input(shape=(1,))
out = Dense(32, activation='relu')(inp)
out = Dense(1)(out)

In [ ]:
with tf.device('/job:ps/task:0'):
    print m.weights[0].device

In [ ]:
print model.input
print model.output

In [ ]:
model = Model(input=inp, output=out)

In [ ]:
model.save('temp.h5')

In [3]:
m = keras.models.load_model('temp.h5')

/usr/local/lib/python2.7/dist-packages/keras/models.py:150: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
m.weights

In [6]:
inp = tf.placeholder(tf.float32, shape=(None, 1))

In [7]:
with tf.device('/job:ps/task:0'):
    #m = keras.models.load_model('temp.h5')
    mm = m(inp)

In [9]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
# for var in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
#     print var.op.name

In [10]:
tf.gradients(mm, m.weights[-1])

[<tf.Tensor 'gradients/add_3_grad/Reshape_1:0' shape=(1,) dtype=float32>]

In [ ]:
with tf.device('/job:localhost/task:0'):
    m = keras.models.load_model('temp.h5')

In [ ]:
with tf.variable_scope('policy11'):
    m = keras.models.load_model('temp.h5')
    for var in m.weights:
        print var.op.name

In [ ]:
m.summary()

In [ ]:
m.layers[-2].output

In [ ]:
m.input

In [ ]:
with tf.variable_scope('policy'):
    out = Dense(128)(m.layers[-2].output)
    new = Model(input=m.layers[0].input, output=out)

In [ ]:
new.summary()

In [ ]:
for var in new.weights:
    print var.op.name

In [ ]:
m.layers[0].input